In [1]:
from src.stepper.serial_utilities.serial_utilities import (

    test_connection,
    print_ports_info,
    scan_ports,
    TestCase,
)

print_ports_info()
test_connection("COM3", 115200, test_case=TestCase.default())
scan_ports(test_case=TestCase.default())


-----------------------------------------------------------------------------------------------------------------------------
Port       Description                    VID   PID   Manufacturer              Serial Number             Location                 
-----------------------------------------------------------------------------------------------------------------------------
COM1       Communications Port (COM1)     N/A   N/A   (Standard port types)     N/A                       N/A                      
COM3       USB-SERIAL CH340 (COM3)        6790  29987 wch.cn                    N/A                       1-8                      
-----------------------------------------------------------------------------------------------------------------------------


{'COM1': 0, 'COM3': 115200}

In [2]:
from serial import Serial
serial_connection = Serial("COM3", 115200, timeout=0.1)


In [6]:
type(serial_connection)


serial.serialwin32.Serial

In [9]:
serial_connection.close()


In [16]:
serial_connection.is_open

False

In [1]:
from src.stepper.device import Device
serial_connection = Device.SerialConnection("COM3", 115200, timeout=0.1)
serial_connection.close()
device_params = Device.DeviceParams(serial_connection=serial_connection, address=1)
device = Device(device_params)

In [2]:
device.position_error

PositionErrorParams(error=0, unit=<AngleUnit.deg: 182.04444444444445>, _value=0.0)

In [4]:
from src.stepper.stepper_core.parameters import (
    JogParams,
    PositionParams,
    StartSpeedParams,
    LoopMode,
    SpeedReduction,
)
(
    jog_params=JogParams(),
    position_params=PositionParams(),
    start_speed_params=StartSpeedParams(),
    loop_mode=LoopMode.CLOSED,
    speed_reduction=SpeedReduction.DISABLE,
)


In [1]:
class a:
    b = 1
    c = 2
    
    @classmethod
    def changeb(cls):
        cls.b = 3
    
print(a.b)
a.changeb()
print(a.b)

1
3


In [5]:
device = Device(device_params=device_params, input_params=input_params)


In [8]:
device.real_time_position.value


0.0494384765625

In [1]:
import time
from serial import Serial

from src.stepper.commands.get import (
    GetBusVoltage,
    GetConfig,
    GetEncoderValue,
    GetPositionError,
    GetMotorRH,
    GetPID,
    GetPhaseCurrent,
    GetRealTimePosition,
    GetRealTimeSpeed,
    GetStatus,
    GetSysStatus,
    GetTargetPosition,
    GetVersion,
)
from src.stepper.commands.home import (
    GetHomeStatus,
    Home,
    RetrieveHomeParam,
    SetHome,
    SetHomeParam,
    StopHome,
)
from src.stepper.commands.move import Disable, EStop, Enable, Jog, Move, SyncMove
from src.stepper.commands.set import (
    Kpid,
    LoopMode,
    Microstep,
    OpenLoopCurrent,
    SetConfig,
    SetID,
    SetLoopMode,
    SetMicrostep,
    SetOpenLoopCurrent,
    SetPID,
    SetReduction,
    SetStartSpeed,
    SpeedReduction,
    StartSpeedParams,
)
from src.stepper.commands.system import CalibrateEncoder, ZeroAllPositions
from src.stepper.stepper_core.configs import (
    AbsoluteFlag,
    Acceleration,
    Address,
    Direction,
    HomingMode,
    PulseCount,
    Speed,
    SyncFlag,
)
from src.stepper.stepper_core.parameters import DeviceParams, JogParams, PositionParams

# from time import sleep
# import logging
# logging.basicConfig(level=logging.DEBUG)

serial_connection = Serial("COM3", 115200, timeout=0.1)

device_params = DeviceParams(
    serial_connection=serial_connection,
    address=Address(0x01),
)

position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(500),
    acceleration=Acceleration(127),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
)

jog_params = JogParams(
    direction=Direction.CW,
    speed=Speed(100),
    acceleration=Acceleration(255),
)

In [2]:
SetHome(device=device_params).status

zero_all_positions = ZeroAllPositions
zero_all_positions.unlock()
zero_all_positions(device=device_params).status


'SUCCESS'

In [2]:
"""Stepper motor class."""

from dataclasses import dataclass
from logging import DEBUG, getLogger
from time import sleep

from serial import Serial

from src.stepper.commands.move import Disable, Enable, EStop, Jog, Move
from src.stepper.stepper_core.configs import (
    AbsoluteFlag,
    Acceleration,
    Address,
    AngleUnit,
    AutoHoming,
    BaudRate,
    CanRate,
    ChecksumMode,
    ClosedLoopCurrent,
    CollisionDetectionCurrent,
    CollisionDetectionSpeed,
    CollisionDetectionTime,
    CommunicationMode,
    ControlMode,
    CurrentUnit,
    DefaultDir,
    Direction,
    EnableFlag,
    EnableLevel,
    EnablePin,
    HomingDirection,
    HomingMode,
    HomingSpeed,
    HomingTimeout,
    InductanceUnit,
    Kpid,
    LoopMode,
    MaxVoltage,
    Microstep,
    MicrostepInterp,
    MotorType,
    OnTargetWindow,
    OpenLoopCurrent,
    PulseCount,
    ResistanceUnit,
    ResponseMode,
    ScreenOff,
    Speed,
    SpeedReduction,
    SpeedUnit,
    StallCurrent,
    StallProtect,
    StallSpeed,
    StallTime,
    StoreFlag,
    SyncFlag,
    TimeUnit,
    VoltageUnit,
)
from src.stepper.stepper_core.parameters import (
    ConfigParams,
    DeviceParams,
    HomingParams,
    JogParams,
    PIDParams,
    PositionParams,
    StartSpeedParams,
)

logger = getLogger(__name__)
logger.setLevel(DEBUG)


@dataclass
class DeviceConfiguration:
    """Parameters and default settings for stepper motor configuration and movement.

    :param speed: Movement speed in RPM (0-3000)
    :param acceleration: Movement acceleration (0-255)
    :param pulse_count: Number of pulses to move (0-4294967295)
    :param direction: Movement direction (CW/CCW)
    :param absolute: Absolute or relative movement (RELATIVE/ABSOLUTE)
    :param sync: Synchronous movement flag (NO_SYNC/SYNC)
    :param microstep: Microstep resolution (0-255)
    :param open_loop_current: Open loop current in mA (0-2000)
    :param closed_loop_current: Closed loop current in mA (0-4000)
    :param loop_mode: Control loop mode (OPEN/CLOSED)
    :param control_mode: Motor control mode (PUL_OFF/PUL_OPEN/PUL_FOC/ESI_RCO)
    :param motor_type: Motor type (D18/D09)
    :param communication_mode: Communication mode (RXTX_OFF/ESI_AL0/UART/CAN)
    :param enable_level: Enable level (LOW/HIGH/HOLD)
    :param microstep_interp: Microstep interpolation (ENABLE/DISABLE)
    :param screen_off: Screen off flag (ENABLE/DISABLE)
    :param max_voltage: Maximum voltage in mV (0-5000)
    :param baud_rate: Serial communication baud rate
    :param can_rate: CAN bus rate
    :param checksum_mode: Checksum mode (FIXED/XOR/CRC8)
    :param response_mode: Response mode (NONE/RECEIVE/REACHED/BOTH/OTHER)
    :param stall_protect: Stall protection flag (ENABLE/DISABLE)
    :param stall_speed: Stall speed in RPM (0-500)
    :param stall_current: Stall current in mA (0-3000)
    :param stall_time: Stall time in ms (0-5000)
    :param on_target_window: Position window in 0.1 degrees (0-100)
    :param enable_pin: Enable pin configuration (ENABLE/DISABLE)
    :param default_dir: Default direction (CW/CCW)
    :param speed_reduction: Speed reduction configuration (ENABLE/DISABLE)
    :param store_flag: Store configuration flag (TEMPORARY/PERMANENT)
    :param enable_flag: Enable/Disable flag
    :param address: Device address (0-255)
    :param kpid: PID control parameters
    :param homing_mode: Homing mode configuration
    :param homing_direction: Homing direction
    :param homing_speed: Homing speed in RPM
    :param homing_timeout: Homing timeout in ms
    :param collision_detection_speed: Collision detection speed in RPM
    :param collision_detection_current: Collision detection current in mA
    :param collision_detection_time: Collision detection time in ms
    :param speed_unit: Speed unit configuration
    :param current_unit: Current unit configuration
    :param voltage_unit: Voltage unit configuration
    :param inductance_unit: Inductance unit configuration
    :param resistance_unit: Resistance unit configuration
    :param angle_unit: Angle unit configuration
    :param time_unit: Time unit configuration
    """

    com_port: str
    baud_rate: BaudRate | int
    address: Address | int

    delay: float | None = None
    # Units configuration
    speed_unit: SpeedUnit = SpeedUnit.RPM
    current_unit: CurrentUnit = CurrentUnit.mA
    voltage_unit: VoltageUnit = VoltageUnit.mV
    inductance_unit: InductanceUnit = InductanceUnit.uH
    resistance_unit: ResistanceUnit = ResistanceUnit.mOhm
    angle_unit: AngleUnit = AngleUnit.deg
    time_unit: TimeUnit = TimeUnit.ms

    # Movement parameters
    speed: Speed = Speed(500)
    acceleration: Acceleration = Acceleration(250)
    pulse_count: PulseCount = PulseCount(80)
    direction: Direction = Direction.CW
    absolute: AbsoluteFlag = AbsoluteFlag.RELATIVE
    sync: SyncFlag = SyncFlag.NO_SYNC

    # Current and resolution
    microstep: Microstep = Microstep(16)
    open_loop_current: OpenLoopCurrent = OpenLoopCurrent(800)
    closed_loop_current: ClosedLoopCurrent = ClosedLoopCurrent(2000)

    # Control modes
    loop_mode: LoopMode = LoopMode.CLOSED
    control_mode: ControlMode = ControlMode.PUL_FOC
    motor_type: MotorType = MotorType.D18
    communication_mode: CommunicationMode = CommunicationMode.UART
    enable_level: EnableLevel = EnableLevel.HOLD

    # Features
    microstep_interp: MicrostepInterp = MicrostepInterp.ENABLE
    screen_off: ScreenOff = ScreenOff.DISABLE
    max_voltage: MaxVoltage = MaxVoltage(4000)

    # Communication
    can_rate: CanRate = CanRate.CAN_500K
    checksum_mode: ChecksumMode = ChecksumMode.FIXED
    response_mode: ResponseMode = ResponseMode.RECEIVE

    # Protection
    stall_protect: StallProtect = StallProtect.ENABLE
    stall_speed: StallSpeed = StallSpeed(28)
    stall_current: StallCurrent = StallCurrent(2400)
    stall_time: StallTime = StallTime(1000)

    # Positioning
    on_target_window: OnTargetWindow = OnTargetWindow(1)
    enable_pin: EnablePin = EnablePin.ENABLE
    default_dir: DefaultDir = DefaultDir.CW
    speed_reduction: SpeedReduction = SpeedReduction.DISABLE

    # System flags
    store_flag: StoreFlag = StoreFlag.TEMPORARY
    enable_flag: EnableFlag = EnableFlag.ENABLE

    # PID Control
    kp: Kpid = Kpid(Kpid.default_kp)
    ki: Kpid = Kpid(Kpid.default_ki)
    kd: Kpid = Kpid(Kpid.default_kd)

    # Homing parameters
    homing_mode: HomingMode = HomingMode.SINGLE_TURN_NEAREST
    homing_direction: HomingDirection = HomingDirection.CW
    homing_speed: HomingSpeed = HomingSpeed(30)
    homing_timeout: HomingTimeout = HomingTimeout(10000)
    collision_detection_speed: CollisionDetectionSpeed = CollisionDetectionSpeed(300)
    collision_detection_current: CollisionDetectionCurrent = CollisionDetectionCurrent(800)
    collision_detection_time: CollisionDetectionTime = CollisionDetectionTime(60)
    auto_home: AutoHoming = AutoHoming.ENABLE

    def __post_init__(self):
        """Post initialization."""
        self.address = Address(self.address)
        self.baud_rate = BaudRate[f"BAUD_{self.baud_rate}"]

    @property
    def serial_connection(self) -> Serial:
        """Serial connection."""
        return Serial(
            port=self.com_port,
            baudrate=int(self.baud_rate),
        )

    @property
    def jog_params(self) -> JogParams:
        """Jog parameters."""
        return JogParams(
            direction=self.direction,
            speed=self.speed,
            acceleration=self.acceleration,
        )

    @property
    def position_params(self) -> PositionParams:
        """Position parameters."""
        return PositionParams(
            direction=self.direction,
            speed=self.speed,
            acceleration=self.acceleration,
            pulse_count=self.pulse_count,
            absolute=self.absolute,
        )

    @property
    def homing_params(self) -> HomingParams:
        """Homing parameters."""
        return HomingParams(
            homing_mode=self.homing_mode,
            homing_direction=self.homing_direction,
            homing_speed=self.homing_speed,
            homing_timeout=self.homing_timeout,
            collision_detection_speed=self.collision_detection_speed,
            collision_detection_current=self.collision_detection_current,
            collision_detection_time=self.collision_detection_time,
            auto_home=self.auto_home,
            current_unit=self.current_unit,
            speed_unit=self.speed_unit,
            time_unit=self.time_unit,
        )

    @property
    def pid_params(self) -> PIDParams:
        """PID parameters."""
        return PIDParams(
            kp=self.kp,
            ki=self.ki,
            kd=self.kd,
        )

    @property
    def start_speed_params(self) -> StartSpeedParams:
        """Start speed parameters."""
        return StartSpeedParams(
            speed=self.speed,
            acceleration=self.acceleration,
            pulse_count=self.pulse_count,
            absolute=self.absolute,
        )

    @property
    def config_params(self) -> ConfigParams:
        """Configuration parameters."""
        return ConfigParams(
            stepper_type=self.motor_type,
            control_mode=self.control_mode,
            communication_mode=self.communication_mode,
            enable_level=self.enable_level,
            default_direction=self.default_dir,
            microsteps=self.microstep,
            microstep_interp=self.microstep_interp,
            screen_off=self.screen_off,
            open_loop_current=self.open_loop_current,
            max_closed_loop_current=self.closed_loop_current,
            max_voltage=self.max_voltage,
            baud_rate=self.baud_rate,
            can_rate=self.can_rate,
            address=self.address,
            checksum_mode=self.checksum_mode,
            response_mode=self.response_mode,
            stall_protect=self.stall_protect,
            stall_speed=self.stall_speed,
            stall_current=self.stall_current,
            stall_time=self.stall_time,
            on_target_window=self.on_target_window,
            current_unit=self.current_unit,
            voltage_unit=self.voltage_unit,
            angle_unit=self.angle_unit,
            time_unit=self.time_unit,
            speed_unit=self.speed_unit,
        )


class Device:
    """Stepper motor class."""

    def __init__(self, config: DeviceConfiguration):
        """Initialize stepper motor."""
        self.config = config
        self.serial_connection = config.serial_connection
        self.device_params = DeviceParams(
            serial_connection=self.serial_connection,
            address=self.config.address,
            checksum_mode=self.config.checksum_mode,
            delay=self.config.delay,
        )

    def enable(self):
        """Enable motor."""
        logger.debug("enabling motor")
        Enable(device=self.device_params)

    def disable(self):
        """Disable motor."""
        logger.debug("disabling motor")
        Disable(device=self.device_params)

    def jog(self):
        """Jog at the configured speed and acceleration."""
        logger.debug("jogging at configured speed and acceleration")
        Jog(device=self.device_params, params=self.config.jog_params)

    def jog_cw(self):
        """Jog clockwise."""
        print(self.serial_connection.is_open)
        params = self.config.jog_params
        params.direction = Direction.CW
        command = Jog(device=self.device_params, params=params)
        print(command.data)

    def jog_ccw(self):
        """Jog counterclockwise."""
        logger.debug("jogging counterclockwise")
        params = self.config.jog_params
        params.direction = Direction.CCW
        Jog(device=self.device_params, params=params)

    def move_cw(self):
        """Move unit distance clockwise."""
        logger.debug("moving unit distance clockwise")
        params = self.config.position_params
        params.direction = Direction.CW
        Move(device=self.device_params, params=params)

    def move_ccw(self):
        """Move unit distance counterclockwise."""
        logger.debug("moving unit distance counterclockwise")
        params = self.config.position_params
        params.direction = Direction.CCW
        Move(device=self.device_params, params=params)

    def move_to(self, position: int):
        """Move to position."""
        logger.debug(f"moving to position {position}")
        params = self.config.position_params
        if position > 0:
            params.direction = Direction.CW
        else:
            params.direction = Direction.CCW
        params.pulse_count = PulseCount(abs(position))
        Move(device=self.device_params, params=params)

    def stop(self):
        """Emergency stop."""
        logger.debug("emergency stop")
        EStop(device=self.device_params)

    def pause(self, time: float = 1):
        """Pause movement."""
        logger.debug(f"pausing for {time} seconds")
        sleep(time)



device_params = DeviceParams(
    serial_connection=serial_connection,
    address=Address(0x01),
)

position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(250),
    acceleration=Acceleration(200),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
    )
    cmd = Enable(device=device_params).status
    cmd = Move(device=device_params, params=position_params).status
    # time.sleep(0.1)
    # print(GetPositionError(device=device_params).raw_data.data_dict)
    # print(GetRealTimePosition(device=device_params).raw_data.data_dict)
    # print(GetTargetPosition(device=device_params).raw_data.data_dict)


In [28]:
device.move()


In [ ]:
de

In [7]:
position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(250),
    acceleration=Acceleration(200),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
)

Enable(device=device_params).status
Move(device=device_params, params=position_params).status
time.sleep(0.1)
print(GetPositionError(device=device_params).raw_data.data_dict)
print(GetRealTimePosition(device=device_params).raw_data.data_dict)
print(GetTargetPosition(device=device_params).raw_data.data_dict)


{'position_error (deg)': 0.560302734375}
{'real_time_position (deg)': 288.1658935546875}
{'target_position (deg)': 292.9449462890625}


In [6]:
Disable(device=device_params).status
Move(device=device_params, params=position_params).status
# This should show a CONDITIONAL_ERROR  

'CONDITIONAL_ERROR'

In [9]:
Jog(device=device_params, params=jog_params).status

'SUCCESS'

In [10]:
EStop(device=device_params).status

'SUCCESS'

In [12]:
Move(device=device_params, params=position_params, setting=SyncFlag.SYNC).is_success
SyncMove(device=device_params).status

'SUCCESS'

In [16]:
position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(100),
    acceleration=Acceleration(255),
    pulse_count=PulseCount(6400),
    absolute=AbsoluteFlag.RELATIVE,
)

SetHome(device=device_params).status
Move(device=device_params, params=position_params).status



'SUCCESS'

In [17]:
Home(device=device_params, params=HomingMode.MULTI_TURN_UNLIMITED).status
# sleep(1)
# StopHome(device=device_params).status

'SUCCESS'

In [19]:
RetrieveHomeParam(device=device_params).data

{'homing_mode': 'SINGLE_TURN_NEAREST',
 'homing_direction': 'CW',
 'homing_speed (RPM)': 30.0,
 'homing_timeout': 10000,
 'collision_detection_speed (RPM)': 300.0,
 'collision_detection_current (mA)': 800.0,
 'collision_detection_time (ms)': 60.0,
 'auto_home': 'DISABLE'}

In [13]:
homing_params = RetrieveHomeParam(device=device_params).raw_data


In [16]:
homing_params.bytes

b"\x00\x00\x00\x1e\x00\x00'\x10\x01,\x03 \x00d\x00"

In [17]:
SetHomeParam(device=device_params, params=homing_params).status

'SUCCESS'

In [16]:
Move(device=device_params, params=position_params).status



'CONDITIONAL_ERROR'

In [17]:
Home(device=device_params, params=HomingMode.SINGLE_TURN_NEAREST).status

'CONDITIONAL_ERROR'

In [22]:
GetHomeStatus(device=device_params).raw_data.data_dict

{'encoder_ready': True,
 'encoder_calibrated': True,
 'homing_active': True,
 'homing_failed': False}

In [27]:

calibrate_encoder = CalibrateEncoder
calibrate_encoder.unlock()
calibrate_encoder(device=device_params).status


'CONDITIONAL_ERROR'

In [23]:

print(GetVersion(device=device_params).data)
print(GetMotorRH(device=device_params).data)
print(GetPID(device=device_params).data)
print(GetConfig(device=device_params).data)
print(GetStatus(device=device_params).data)
print(GetPositionError(device=device_params).data)
print(GetSysStatus(device=device_params).data)
print(GetBusVoltage(device=device_params).data)
print(GetPhaseCurrent(device=device_params).data)
print(GetEncoderValue(device=device_params).data)



{'firmware_version': 255, 'hardware_version': 120}
{'phase_resistance (mOhm)': 3417.0, 'phase_inductance (uH)': 1160.0}
{'pid_p': 32000, 'pid_i': 10, 'pid_d': 32000}
{'stepper_type': 'D18', 'control_mode': 'PUL_FOC', 'communication_mode': 'UART', 'enable_level': 'HOLD', 'default_direction': 'CW', 'microsteps': 16, 'microstep_interp': 'ENABLE', 'screen_off': 'DISABLE', 'open_loop_current (mA)': 800.0, 'max_closed_loop_current (mA)': 3000.0, 'max_voltage (mV)': 5000.0, 'baud_rate': 'BAUD_115200', 'can_rate': 'CAN_500K', 'address': 1, 'checksum_mode': 'FIXED', 'response_mode': 'RECEIVE', 'stall_protect': 'ENABLE', 'stall_speed (RPM)': 8.0, 'stall_current (mA)': 2600.0, 'stall_time (ms)': 2000.0, 'on_target_window': 0.3}
{'enabled': True, 'in_position': True, 'stalled': False, 'stall_protection_active': False}
{'position_error (deg)': 0.0}
{'bus_voltage (mV)': 11934.0, 'bus_phase_current (mA)': 8.0, 'calibrated_encoder_value (deg)': 14.56787109375, 'stepper_target_position (deg)': 0.0, 'st

In [24]:
from itertools import chain

sys_status = GetSysStatus(device=device_params).raw_data
config_params = GetConfig(device=device_params).raw_data
for key, value in sorted(chain(sys_status.data_dict.items(), config_params.data_dict.items())):
    print(f"{key}: {value}")


address: 1
baud_rate: BAUD_115200
bus_phase_current (mA): 28.0
bus_voltage (mV): 11924.0
calibrated_encoder_value (deg): 14.56787109375
can_rate: CAN_500K
checksum_mode: FIXED
communication_mode: UART
control_mode: PUL_FOC
default_direction: CW
enable_level: HOLD
encoder_calibrated: True
encoder_ready: True
homing_failed: False
is_homing: True
max_closed_loop_current (mA): 3000.0
max_voltage (mV): 5000.0
microstep_interp: ENABLE
microsteps: 16
on_target_window: 0.3
open_loop_current (mA): 800.0
response_mode: RECEIVE
screen_off: DISABLE
stall_current (mA): 2600.0
stall_protect: ENABLE
stall_speed (RPM): 8.0
stall_time (ms): 2000.0
stepper_enabled: True
stepper_in_position: True
stepper_position_error (deg): 0.0
stepper_real_time_position (deg): 0.0
stepper_real_time_speed (RPM): 0.0
stepper_stall_protection_active: False
stepper_stalled: False
stepper_target_position (deg): 0.0
stepper_type: D18


In [38]:
Set_Microstep = SetMicrostep
Set_Microstep.unlock()
Set_Microstep(device=device_params, params=Microstep(value=16)).status

'SUCCESS'

In [39]:
Set_ID = SetID
Set_ID.unlock()
Set_ID(device=device_params, params=Address(value=1)).status


'SUCCESS'

In [25]:
SetLoopMode = SetLoopMode
SetLoopMode.unlock()
SetLoopMode(device=device_params, params=LoopMode.CLOSED).status


'SUCCESS'

In [41]:
Set_open_loop_current = SetOpenLoopCurrent
Set_open_loop_current.unlock()
Set_open_loop_current(device=device_params, params=OpenLoopCurrent(value=800)).status

'SUCCESS'

In [42]:
PID_params = GetPID(device=device_params).raw_data
print(PID_params.data_dict)
Set_PID = SetPID
Set_PID.unlock()
Set_PID(device=device_params, params=PID_params).status


{'pid_p': 32000, 'pid_i': 10, 'pid_d': 32000}


'SUCCESS'

In [43]:
from src.stepper.stepper_constants import EnablePin
start_speed_params = StartSpeedParams(
    direction=Direction.CW,
    speed=Speed(1000),
    acceleration=Acceleration(255),
    en_control=EnablePin.ENABLE,
)
Set_StartSpeed = SetStartSpeed
Set_StartSpeed.unlock()
Set_StartSpeed(device=device_params, params=start_speed_params).status

'SUCCESS'

In [26]:
from src.stepper.stepper_core.configs import StallTime

config_params = GetConfig(device=device_params).raw_data
print("Initial Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

config_params.microstep = Microstep(value=32)
config_params.stall_time = StallTime(value=100)

Set_Config = SetConfig
Set_Config.unlock()
Set_Config(device=device_params, params=config_params).status

config_params = GetConfig(device=device_params).raw_data
print("-----------------------")
print("Changed Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

config_params.microstep = Microstep(Microstep.default)
config_params.stall_time = StallTime(StallTime.default)

Set_Config = SetConfig
Set_Config.unlock()
Set_Config(device=device_params, params=config_params).status


config_params = GetConfig(device=device_params).raw_data
print("-----------------------")
print("Changed Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")



Initial Config:
address: 1
baud_rate: BAUD_115200
can_rate: CAN_500K
checksum_mode: FIXED
communication_mode: UART
control_mode: PUL_FOC
default_direction: CW
enable_level: HOLD
max_closed_loop_current (mA): 3000.0
max_voltage (mV): 5000.0
microstep_interp: ENABLE
microsteps: 16
on_target_window: 0.3
open_loop_current (mA): 800.0
response_mode: RECEIVE
screen_off: DISABLE
stall_current (mA): 2600.0
stall_protect: ENABLE
stall_speed (RPM): 8.0
stall_time (ms): 2000.0
stepper_type: D18
-----------------------
Changed Config:
address: 1
baud_rate: BAUD_115200
can_rate: CAN_500K
checksum_mode: FIXED
communication_mode: UART
control_mode: PUL_FOC
default_direction: CW
enable_level: HOLD
max_closed_loop_current (mA): 3000.0
max_voltage (mV): 5000.0
microstep_interp: ENABLE
microsteps: 16
on_target_window: 0.3
open_loop_current (mA): 800.0
response_mode: RECEIVE
screen_off: DISABLE
stall_current (mA): 2600.0
stall_protect: ENABLE
stall_speed (RPM): 8.0
stall_time (ms): 100.0
stepper_type: D18